# Order Flow and Advanced Neural Networks

By Aric Cutuli<br>
***

## Introduction

The enigma concerning the predictability of markets has always been the principal driver of my interest in finance, and it inpires my ongoing exploration of machine learning's applications within financial modeling. Today, we delve into recent literary discussions concerning the intersection of supervised learning and limit order book forecasting.

In this notebook, we implement an advanced neural network originally employed by (Zhang et al) that combines convolutional neural networks (CNNs) and a long-short term memory network (LSTM) in order to classify future directions of an order book at a high frequency. Unlike the works of (Zhang et al), which use raw nonstationary order book states as the input to the network, our instantiation of the architecture is trained on order flow, which are stationary quantities derived from the limit order book (Cont et al). Hence, this discussion also draws heavy inspiration from a recent article (Kolm et al) that uses order flow and supervised learning to formulate the order book forecasting problem as one of regression.

To extend the discussions of the aforementioned papers, we build a trading signal based off of the trained network.

## Limit Order Book and Order Flow

Modern equity trading is conducted electronically and is often facilitated by a limit order book. The order book collects bids and offers made by prospective buyers and sellers. The bid price is the highest price buyers are prepared to buy at, and the ask price is the lowest price sellers are willing to sell at. An order is submitted with the inputs side, quantity, price, and time.

## Our Forecasting Model

Producing a forecast is simply a matter of taking the conditional expectation of the data under the model.

Feature extraction of microstructure data has generically been handled with PCA or LDA, but here we adopt CNNs and Inception Modules to accomplish this task.

CNNs are used to automate the feature extraction process instead of feature engineering and PCA. (Zhang et al)

CNNs attempt to reduce the network size by exploiting data locality (Dixon + Halperin)

in financial modeling, we typically have different spatial structures, such as the limit order book depths

Inception module increases width and depth of the convolutions, hence helps capture local interactions bewtween timestamps. (Szegedy et al) Infer decay rates of the autoregressive process. (Zhang et al)

## Data

In [1]:
pip install cbpro

Note: you may need to restart the kernel to use updated packages.


In [2]:
from cbpro import PublicClient
import pandas as pd
from time import time, strftime, gmtime
import numpy as np

In [16]:
public_client = PublicClient()
dataframe = pd.DataFrame()
start = time()
while len(dataframe) < 110:
    raw_data = pd.concat((pd.DataFrame.from_dict(public_client.get_product_order_book('BTC-USD',level=2)['asks'])[:10],
                     pd.DataFrame.from_dict(public_client.get_product_order_book('BTC-USD',level=2)['bids'])[:10]),axis=1)
    dataframe = pd.concat((dataframe, pd.concat((pd.DataFrame(raw_data.drop(2,axis=1).iloc[i]).T for i in range(10)), axis=1).apply(lambda x: pd.Series(x.dropna().values))))
end = time()
print((end-start)/len(dataframe))

0.6848138830878517


In [17]:
dataframe

0           1         0           1         0           1  \
0   29645.23  0.03373845  29645.13  0.04803743  29648.51       0.093   
0   29645.23  0.03373845  29645.13  0.04803743  29648.51       0.093   
0   29648.58   0.0444414  29648.57  0.40187743  29648.59  0.21853172   
0   29648.58   0.0444414  29648.57  0.40187743  29648.59  0.21853172   
0   29648.58   0.0444414  29656.11  0.20137193  29648.59  0.21853172   
..       ...         ...       ...         ...       ...         ...   
0   29657.83     0.43584  29657.82  0.10119065  29657.89       0.001   
0   29657.83     0.43584  29657.82  0.10119065  29657.89       0.001   
0   29657.92       0.001  29657.91  0.15758612  29657.93  0.02390464   
0   29657.92       0.001  29657.91  0.15758612  29657.93  0.02390464   
0   29657.92       0.001  29657.91  0.15758612  29657.93  0.02390464   

           0           1         0           1  ...         0           1  \
0   29644.18  0.02439493  29648.59  0.16853172  ...  29639.99  0.00405194   
0   29644.18  0.02439493  29648.59  0.16853172  ...  29639.99  0.00405194   
0   29646.57  0.00203743  29650.84  0.03373215  ...  29642.31  0.00271233   
0   29646.57  0.00203743  29650.84  0.03373215  ...  29642.31  0.00271233   
0   29654.64    0.028224  29650.84  0.03373215  ...  29650.18    0.001686   
..       ...         ...       ...         ...  ...       ...         ...   
0   29655.35  0.02709227   29657.9       0.001  ...  29654.67  0.02446132   
0   29655.35  0.02709227   29657.9       0.001  ...  29654.67  0.02446132   
0   29657.82  0.00005004  29659.66        0.05  ...  29655.22  0.03198692   
0   29657.82  0.00005004  29659.66        0.05  ...  29655.22  0.03198692   
0   29657.82  0.00005004  29659.66        0.05  ...  29655.22  0.03198692   

           0         1         0           1         0           1         0  \
0   29651.39       0.1   29639.2  0.58363217   29651.7  0.01581346  29639.01   
0   29651.39       0.1   29639.2  0.58363217   29651.7  0.01581346  29639.01   
0   29652.32     0.235   29642.3  0.17948003  29652.42  0.01155213  29642.29   
0   29652.32     0.235   29642.3  0.17948003  29652.42  0.01155213  29642.29   
0   29652.32     0.235  29649.28   0.0129926  29652.42  0.01155213  29649.08   
..       ...       ...       ...         ...       ...         ...       ...   
0   29659.71  0.001686  29654.65   0.0167918  29659.84  0.00158055   29654.6   
0   29659.71  0.001686  29654.65   0.0167918  29659.84  0.00158055   29654.6   
0    29660.5      0.02  29655.01  0.01814594  29660.66       0.049  29654.92   
0    29660.5      0.02  29655.01  0.01814594  29660.66       0.049  29654.92   
0    29660.5      0.02  29655.01  0.01814594  29660.66       0.049  29654.92   

             1  
0   0.01497027  
0   0.01497027  
0   0.02414143  
0   0.02414143  
0   0.17944799  
..         ...  
0   0.02847664  
0   0.02847664  
0   0.02701418  
0   0.02701418  
0   0.02701418  

[110 rows x 40 columns]

In [18]:
dataframe.columns = ['PRICE_ASK_0','VOLUME_ASK_0','PRICE_BID_0','VOLUME_BID_0',
           'PRICE_ASK_1','VOLUME_ASK_1','PRICE_BID_1','VOLUME_BID_1',
           'PRICE_ASK_2','VOLUME_ASK_2','PRICE_BID_2','VOLUME_BID_2',
           'PRICE_ASK_3','VOLUME_ASK_3','PRICE_BID_3','VOLUME_BID_3',
           'PRICE_ASK_4','VOLUME_ASK_4','PRICE_BID_4','VOLUME_BID_4',
           'PRICE_ASK_5','VOLUME_ASK_5','PRICE_BID_5','VOLUME_BID_5',
           'PRICE_ASK_6','VOLUME_ASK_6','PRICE_BID_6','VOLUME_BID_6',
           'PRICE_ASK_7','VOLUME_ASK_7','PRICE_BID_7','VOLUME_BID_7',
           'PRICE_ASK_8','VOLUME_ASK_8','PRICE_BID_8','VOLUME_BID_8',
           'PRICE_ASK_9','VOLUME_ASK_9','PRICE_BID_9','VOLUME_BID_9']
dataframe.index = range(len(dataframe))
dataframe = dataframe.astype(float)
dataframe

PRICE_ASK_0  VOLUME_ASK_0  PRICE_BID_0  VOLUME_BID_0  PRICE_ASK_1  \
0       29645.23      0.033738     29645.13      0.048037     29648.51   
1       29645.23      0.033738     29645.13      0.048037     29648.51   
2       29648.58      0.044441     29648.57      0.401877     29648.59   
3       29648.58      0.044441     29648.57      0.401877     29648.59   
4       29648.58      0.044441     29656.11      0.201372     29648.59   
..           ...           ...          ...           ...          ...   
105     29657.83      0.435840     29657.82      0.101191     29657.89   
106     29657.83      0.435840     29657.82      0.101191     29657.89   
107     29657.92      0.001000     29657.91      0.157586     29657.93   
108     29657.92      0.001000     29657.91      0.157586     29657.93   
109     29657.92      0.001000     29657.91      0.157586     29657.93   

     VOLUME_ASK_1  PRICE_BID_1  VOLUME_BID_1  PRICE_ASK_2  VOLUME_ASK_2  ...  \
0        0.093000     29644.18      0.024395     29648.59      0.168532  ...   
1        0.093000     29644.18      0.024395     29648.59      0.168532  ...   
2        0.218532     29646.57      0.002037     29650.84      0.033732  ...   
3        0.218532     29646.57      0.002037     29650.84      0.033732  ...   
4        0.218532     29654.64      0.028224     29650.84      0.033732  ...   
..            ...          ...           ...          ...           ...  ...   
105      0.001000     29655.35      0.027092     29657.90      0.001000  ...   
106      0.001000     29655.35      0.027092     29657.90      0.001000  ...   
107      0.023905     29657.82      0.000050     29659.66      0.050000  ...   
108      0.023905     29657.82      0.000050     29659.66      0.050000  ...   
109      0.023905     29657.82      0.000050     29659.66      0.050000  ...   

     PRICE_BID_7  VOLUME_BID_7  PRICE_ASK_8  VOLUME_ASK_8  PRICE_BID_8  \
0       29639.99      0.004052     29651.39      0.100000     29639.20   
1       29639.99      0.004052     29651.39      0.100000     29639.20   
2       29642.31      0.002712     29652.32      0.235000     29642.30   
3       29642.31      0.002712     29652.32      0.235000     29642.30   
4       29650.18      0.001686     29652.32      0.235000     29649.28   
..           ...           ...          ...           ...          ...   
105     29654.67      0.024461     29659.71      0.001686     29654.65   
106     29654.67      0.024461     29659.71      0.001686     29654.65   
107     29655.22      0.031987     29660.50      0.020000     29655.01   
108     29655.22      0.031987     29660.50      0.020000     29655.01   
109     29655.22      0.031987     29660.50      0.020000     29655.01   

     VOLUME_BID_8  PRICE_ASK_9  VOLUME_ASK_9  PRICE_BID_9  VOLUME_BID_9  
0        0.583632     29651.70      0.015813     29639.01      0.014970  
1        0.583632     29651.70      0.015813     29639.01      0.014970  
2        0.179480     29652.42      0.011552     29642.29      0.024141  
3        0.179480     29652.42      0.011552     29642.29      0.024141  
4        0.012993     29652.42      0.011552     29649.08      0.179448  
..            ...          ...           ...          ...           ...  
105      0.016792     29659.84      0.001581     29654.60      0.028477  
106      0.016792     29659.84      0.001581     29654.60      0.028477  
107      0.018146     29660.66      0.049000     29654.92      0.027014  
108      0.018146     29660.66      0.049000     29654.92      0.027014  
109      0.018146     29660.66      0.049000     29654.92      0.027014  

[110 rows x 40 columns]

 we extract labels based on short-term
and long-term, event -based, relative changes for the next 1, 2, 3, 5, and 10
events for our representations dataset.

Our labels describe the percentage change of the mid-price

The extracted labels are based
on a threshold for the percentage change of 0.002. For percentage changes equal
to or greater than 0.002, we use label 1. For percentage change that varies from
-0.00199 to 0.00199, we use label 0, and, for percentage change smaller or equal
to -0.002, we use label -1.

In [19]:
for i in range(len(dataframe)-10):
    for tick in (1,2,3,5,10):
        dataframe['LABEL_{}TICK'.format(tick)] = np.zeros(len(dataframe))
        if (dataframe['PRICE_ASK_0'][i+tick] + dataframe['PRICE_BID_0'][i+tick]) / 2 > 1.00002*(dataframe['PRICE_ASK_0'][i] + dataframe['PRICE_BID_0'][i]) / 2:
            dataframe['LABEL_{}TICK'.format(tick)][i] = 1
        elif (dataframe['PRICE_ASK_0'][i+tick] + dataframe['PRICE_BID_0'][i+tick]) / 2 < 0.99998*(dataframe['PRICE_ASK_0'][i] + dataframe['PRICE_BID_0'][i]) / 2:
            dataframe['LABEL_{}TICK'.format(tick)][i] = -1
dataframe = dataframe.head(len(dataframe)-10)

In [20]:
dataframe

PRICE_ASK_0  VOLUME_ASK_0  PRICE_BID_0  VOLUME_BID_0  PRICE_ASK_1  \
0      29645.23      0.033738     29645.13      0.048037     29648.51   
1      29645.23      0.033738     29645.13      0.048037     29648.51   
2      29648.58      0.044441     29648.57      0.401877     29648.59   
3      29648.58      0.044441     29648.57      0.401877     29648.59   
4      29648.58      0.044441     29656.11      0.201372     29648.59   
..          ...           ...          ...           ...          ...   
95     29657.80      0.003858     29657.79      0.146110     29657.93   
96     29657.80      0.003858     29657.79      0.146110     29657.93   
97     29657.80      0.003858     29657.79      0.100000     29657.93   
98     29657.80      0.000505     29657.79      0.100000     29657.93   
99     29657.80      0.000505     29657.79      0.100000     29657.93   

    VOLUME_ASK_1  PRICE_BID_1  VOLUME_BID_1  PRICE_ASK_2  VOLUME_ASK_2  ...  \
0       0.093000     29644.18      0.024395     29648.59      0.168532  ...   
1       0.093000     29644.18      0.024395     29648.59      0.168532  ...   
2       0.218532     29646.57      0.002037     29650.84      0.033732  ...   
3       0.218532     29646.57      0.002037     29650.84      0.033732  ...   
4       0.218532     29654.64      0.028224     29650.84      0.033732  ...   
..           ...          ...           ...          ...           ...  ...   
95      0.031967     29657.28      0.015925     29659.60      0.050000  ...   
96      0.031967     29657.28      0.015925     29659.60      0.050000  ...   
97      0.031967     29655.70      0.028681     29659.60      0.050000  ...   
98      0.031967     29655.70      0.028681     29659.60      0.050000  ...   
99      0.031967     29655.70      0.028681     29659.60      0.050000  ...   

    VOLUME_BID_8  PRICE_ASK_9  VOLUME_ASK_9  PRICE_BID_9  VOLUME_BID_9  \
0       0.583632     29651.70      0.015813     29639.01      0.014970   
1       0.583632     29651.70      0.015813     29639.01      0.014970   
2       0.179480     29652.42      0.011552     29642.29      0.024141   
3       0.179480     29652.42      0.011552     29642.29      0.024141   
4       0.012993     29652.42      0.011552     29649.08      0.179448   
..           ...          ...           ...          ...           ...   
95      0.024158     29660.29      0.168630     29654.67      0.024461   
96      0.024158     29660.29      0.168630     29654.67      0.024461   
97      0.024461     29660.29      0.168630     29654.65      0.016792   
98      0.024461     29660.50      0.020000     29654.65      0.016792   
99      0.024461     29660.50      0.020000     29654.65      0.016792   

    LABEL_1TICK  LABEL_2TICK  LABEL_3TICK  LABEL_5TICK  LABEL_10TICK  
0           0.0          0.0          0.0          0.0           0.0  
1           0.0          0.0          0.0          0.0           0.0  
2           0.0          0.0          0.0          0.0           0.0  
3           0.0          0.0          0.0          0.0           0.0  
4           0.0          0.0          0.0          0.0           0.0  
..          ...          ...          ...          ...           ...  
95          0.0          0.0          0.0          0.0           0.0  
96          0.0          0.0          0.0          0.0           0.0  
97          0.0          0.0          0.0          0.0           0.0  
98          0.0          0.0          0.0          0.0           0.0  
99          0.0          0.0          0.0          0.0           0.0  

[100 rows x 45 columns]

In [21]:
dataframe.to_csv('BTC-USD Data {}'.format(strftime('%d %b %Y', gmtime())), index=False)

In [25]:
dataframe = pd.read_csv('BTC-USD Data 02 Jun 2022')
dataframe

PRICE_ASK_0  VOLUME_ASK_0  PRICE_BID_0  VOLUME_BID_0  PRICE_ASK_1  \
0      29645.23      0.033738     29645.13      0.048037     29648.51   
1      29645.23      0.033738     29645.13      0.048037     29648.51   
2      29648.58      0.044441     29648.57      0.401877     29648.59   
3      29648.58      0.044441     29648.57      0.401877     29648.59   
4      29648.58      0.044441     29656.11      0.201372     29648.59   
..          ...           ...          ...           ...          ...   
95     29657.80      0.003858     29657.79      0.146110     29657.93   
96     29657.80      0.003858     29657.79      0.146110     29657.93   
97     29657.80      0.003858     29657.79      0.100000     29657.93   
98     29657.80      0.000505     29657.79      0.100000     29657.93   
99     29657.80      0.000505     29657.79      0.100000     29657.93   

    VOLUME_ASK_1  PRICE_BID_1  VOLUME_BID_1  PRICE_ASK_2  VOLUME_ASK_2  ...  \
0       0.093000     29644.18      0.024395     29648.59      0.168532  ...   
1       0.093000     29644.18      0.024395     29648.59      0.168532  ...   
2       0.218532     29646.57      0.002037     29650.84      0.033732  ...   
3       0.218532     29646.57      0.002037     29650.84      0.033732  ...   
4       0.218532     29654.64      0.028224     29650.84      0.033732  ...   
..           ...          ...           ...          ...           ...  ...   
95      0.031967     29657.28      0.015925     29659.60      0.050000  ...   
96      0.031967     29657.28      0.015925     29659.60      0.050000  ...   
97      0.031967     29655.70      0.028681     29659.60      0.050000  ...   
98      0.031967     29655.70      0.028681     29659.60      0.050000  ...   
99      0.031967     29655.70      0.028681     29659.60      0.050000  ...   

    VOLUME_BID_8  PRICE_ASK_9  VOLUME_ASK_9  PRICE_BID_9  VOLUME_BID_9  \
0       0.583632     29651.70      0.015813     29639.01      0.014970   
1       0.583632     29651.70      0.015813     29639.01      0.014970   
2       0.179480     29652.42      0.011552     29642.29      0.024141   
3       0.179480     29652.42      0.011552     29642.29      0.024141   
4       0.012993     29652.42      0.011552     29649.08      0.179448   
..           ...          ...           ...          ...           ...   
95      0.024158     29660.29      0.168630     29654.67      0.024461   
96      0.024158     29660.29      0.168630     29654.67      0.024461   
97      0.024461     29660.29      0.168630     29654.65      0.016792   
98      0.024461     29660.50      0.020000     29654.65      0.016792   
99      0.024461     29660.50      0.020000     29654.65      0.016792   

    LABEL_1TICK  LABEL_2TICK  LABEL_3TICK  LABEL_5TICK  LABEL_10TICK  
0           0.0          0.0          0.0          0.0           0.0  
1           0.0          0.0          0.0          0.0           0.0  
2           0.0          0.0          0.0          0.0           0.0  
3           0.0          0.0          0.0          0.0           0.0  
4           0.0          0.0          0.0          0.0           0.0  
..          ...          ...          ...          ...           ...  
95          0.0          0.0          0.0          0.0           0.0  
96          0.0          0.0          0.0          0.0           0.0  
97          0.0          0.0          0.0          0.0           0.0  
98          0.0          0.0          0.0          0.0           0.0  
99          0.0          0.0          0.0          0.0           0.0  

[100 rows x 45 columns]

In [3]:
pip install git+https://github.com/simaki/fi2010

  Cloning https://github.com/simaki/fi2010 to /private/var/folders/rz/7bnq_1d91s90b27nyrc7yt5m0000gn/T/pip-req-build-qrl_e89v
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for fi2010: filename=fi2010-0.0.0-py3-none-any.whl size=4504 sha256=a3276a258618d169ee20e944d8b7a97d9df05077832ff014148db5a494ebd058
  Stored in directory: /private/var/folders/rz/7bnq_1d91s90b27nyrc7yt5m0000gn/T/pip-ephem-wheel-cache-qfncc15c/wheels/ae/5a/8b/eea6c3c4724b75002637f4a0b40082248098fdd10bef4f0f13
Successfully built fi2010
Note: you may need to restart the kernel to use updated packages.


In [4]:
from fi2010 import fetch_fi2010
dataframe = fetch_fi2010()

4 of the 5 stocks suffer from a lack of data, so we choose to train our model on the stock with the most data, which reflects liquidity

In [5]:
dataframe = dataframe.iloc[:len(dataframe[dataframe['STOCK']==5])]

In [6]:
columns = ['PRICE_ASK_0','VOLUME_ASK_0','PRICE_BID_0','VOLUME_BID_0',
           'PRICE_ASK_1','VOLUME_ASK_1','PRICE_BID_1','VOLUME_BID_1',
           'PRICE_ASK_2','VOLUME_ASK_2','PRICE_BID_2','VOLUME_BID_2',
           'PRICE_ASK_3','VOLUME_ASK_3','PRICE_BID_3','VOLUME_BID_3',
           'PRICE_ASK_4','VOLUME_ASK_4','PRICE_BID_4','VOLUME_BID_4',
           'PRICE_ASK_5','VOLUME_ASK_5','PRICE_BID_5','VOLUME_BID_5',
           'PRICE_ASK_6','VOLUME_ASK_6','PRICE_BID_6','VOLUME_BID_6',
           'PRICE_ASK_7','VOLUME_ASK_7','PRICE_BID_7','VOLUME_BID_7',
           'PRICE_ASK_8','VOLUME_ASK_8','PRICE_BID_8','VOLUME_BID_8',
           'PRICE_ASK_9','VOLUME_ASK_9','PRICE_BID_9','VOLUME_BID_9',
           'LABEL_1TICK','LABEL_2TICK','LABEL_3TICK','LABEL_5TICK','LABEL_10TICK']
dataframe = dataframe[columns]
dataframe

PRICE_ASK_0  VOLUME_ASK_0  PRICE_BID_0  VOLUME_BID_0  PRICE_ASK_1  \
0             2615           353         2606           326         2618   
1             2615           211         2606           326         2619   
2             2614           122         2606           326         2615   
3             2614           322         2606           326         2617   
4             2614           322         2606           326         2617   
...            ...           ...          ...           ...          ...   
76461         1780          2000         1778           391         1781   
76462         1780           300         1778           391         1781   
76463         1781          3228         1779           300         1782   
76464         1781          3028         1779           300         1782   
76465         1779            49         1778           391         1780   

       VOLUME_ASK_1  PRICE_BID_1  VOLUME_BID_1  PRICE_ASK_2  VOLUME_ASK_2  \
0               200         2604           682         2619           164   
1               164         2604           682         2620           138   
2               200         2604           682         2617           361   
3               938         2604           682         2619           850   
4               938         2604           682         2619           850   
...             ...          ...           ...          ...           ...   
76461          3728         1777          3955         1782          3260   
76462          3728         1777          3955         1782          4260   
76463          4260         1778           391         1783          9970   
76464          4260         1778           391         1783          9470   
76465           500         1777          3955         1781          2008   

       ...  VOLUME_BID_8  PRICE_ASK_9  VOLUME_ASK_9  PRICE_BID_9  \
0      ...           123         2633           311         2579   
1      ...           134         2646           138         2588   
2      ...           134         2633           311         2588   
3      ...           123         2646           138         2579   
4      ...           123         2646           138         2579   
...    ...           ...          ...           ...          ...   
76461  ...          3187         1789          6000         1769   
76462  ...          3187         1789          6000         1769   
76463  ...          1000         1790          1000         1770   
76464  ...          1000         1790          1000         1770   
76465  ...          3187         1788          8160         1769   

       VOLUME_BID_9  LABEL_1TICK  LABEL_2TICK  LABEL_3TICK  LABEL_5TICK  \
0               128            0            0            0            0   
1               123            0            0            0            0   
2               123           -1           -1            0            0   
3               128            0            0           -1            0   
4               128            1            1            1            0   
...             ...          ...          ...          ...          ...   
76461          5864            1            1            1            1   
76462          5864            1            1            1            1   
76463          3187            1            1            1            1   
76464          3187            1           -1            1            1   
76465          5864           -1           -1           -1            1   

       LABEL_10TICK  
0                 0  
1                 0  
2                 0  
3                 0  
4                 0  
...             ...  
76461             1  
76462             1  
76463             1  
76464             1  
76465             1  

[76466 rows x 45 columns]

For our stock, we let t ∈ {1, . . . , T } denote an enumeration of all order book updates. Given two consecutive order book states for the stock at t − 1 and t, we define the bid order flows and ask order flows at time t as vectors

In [7]:
import numpy as np

In [8]:
of_data = pd.DataFrame()
for i in range(10):
    of_data['bOF_{}'.format(i)] = np.empty(len(dataframe))
    of_data['aOF_{}'.format(i)] = np.empty(len(dataframe))
    of_data['bOF_{}'.format(i)][0] = None
    of_data['aOF_{}'.format(i)][0] = None
    for j in range(1,len(dataframe)):
            
        # Bid Order Flow
        if dataframe.loc[j,'PRICE_BID_{}'.format(i)] > dataframe.loc[j-1,'PRICE_BID_{}'.format(i)]:
            of_data['bOF_{}'.format(i)][j] = dataframe.loc[j,'VOLUME_BID_{}'.format(i)]
        elif dataframe.loc[j,'PRICE_BID_{}'.format(i)] < dataframe.loc[j-1,'PRICE_BID_{}'.format(i)]:
            of_data['bOF_{}'.format(i)][j] = -1*dataframe.loc[j,'VOLUME_BID_{}'.format(i)]
        else:
            of_data['bOF_{}'.format(i)][j] = dataframe.loc[j,'VOLUME_BID_{}'.format(i)] - dataframe.loc[j-1,'VOLUME_BID_{}'.format(i)]
            
        # Ask Order Flow
        if dataframe.loc[j,'PRICE_ASK_{}'.format(i)] > dataframe.loc[j-1,'PRICE_ASK_{}'.format(i)]:
            of_data['aOF_{}'.format(i)][j] = -1*dataframe.loc[j,'VOLUME_ASK_{}'.format(i)]
        elif dataframe.loc[j,'PRICE_ASK_{}'.format(i)] < dataframe.loc[j-1,'PRICE_ASK_{}'.format(i)]:
            of_data['aOF_{}'.format(i)][j] = dataframe.loc[j,'VOLUME_ASK_{}'.format(i)]
        else:
            of_data['aOF_{}'.format(i)][j] = dataframe.loc[j,'VOLUME_ASK_{}'.format(i)] - dataframe.loc[j-1,'VOLUME_ASK_{}'.format(i)]

In [9]:
of_data

bOF_0   aOF_0   bOF_1   aOF_1   bOF_2   aOF_2   bOF_3   aOF_3   bOF_4  \
0        NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
1        0.0  -142.0     0.0  -164.0     0.0  -138.0   393.0  -545.0   500.0   
2        0.0   122.0     0.0   200.0     0.0   361.0     0.0    63.0     0.0   
3        0.0   200.0     0.0  -938.0     0.0  -850.0  -893.0  -532.0  -159.0   
4        0.0     0.0     0.0     0.0     0.0     0.0     0.0  -394.0     0.0   
...      ...     ...     ...     ...     ...     ...     ...     ...     ...   
76461    0.0 -2000.0     0.0 -3728.0     0.0 -3260.0     0.0 -8470.0 -2000.0   
76462    0.0 -1700.0     0.0     0.0     0.0  1000.0   500.0  1500.0   500.0   
76463  300.0 -3228.0   391.0 -4260.0  3955.0 -9970.0  1000.0 -4812.0  4839.0   
76464    0.0  -200.0     0.0     0.0     0.0  -500.0     0.0  2000.0     0.0   
76465 -391.0    49.0 -3955.0   500.0 -1000.0  2008.0 -4839.0  3760.0 -4066.0   

         aOF_4   bOF_5   aOF_5   bOF_6   aOF_6   bOF_7   aOF_7   bOF_8  \
0          NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
1       -150.0   159.0  -787.0   100.0  -146.0   143.0  -311.0   134.0   
2        138.0     0.0   545.0     0.0   150.0     0.0   787.0     0.0   
3       -151.0  -100.0  -150.0  -143.0  -146.0  -134.0  -311.0  -123.0   
4        394.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
...        ...     ...     ...     ...     ...     ...     ...     ...   
76461  -4812.0     0.0 -6217.0     0.0 -1897.0     0.0 -1082.0     0.0   
76462      0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
76463  -6217.0  4066.0 -1897.0  3005.0 -1082.0  2162.0 -8160.0  1000.0   
76464      0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
76465  10990.0 -3005.0  6312.0 -2162.0  6217.0 -1000.0  1897.0 -3187.0   

        aOF_8   bOF_9   aOF_9  
0         NaN     NaN     NaN  
1      -165.0   123.0  -138.0  
2       146.0     0.0   311.0  
3      -165.0  -128.0  -138.0  
4         0.0     0.0     0.0  
...       ...     ...     ...  
76461 -8160.0     0.0 -6000.0  
76462     0.0     0.0     0.0  
76463 -6000.0  3187.0 -1000.0  
76464     0.0     0.0     0.0  
76465  1082.0 -5864.0  8160.0  

[76466 rows x 20 columns]

In [10]:
# Add output columns to of_data
of_data = pd.concat([of_data,dataframe.iloc[:,-5:]],axis=1)

In [11]:
# Drop first row
of_data = of_data.iloc[1: , :]

## Methodology

### Analyzing Stationarity

Staionary time series is one such that its statistical parameters such as mean and variance do not change over time. If the linear combination or spread of two time series is stationary, then the two variables are said to be cointegrated. That is, for two time series, if the spread stays around the mean, then the variables are said to be cointegrated. Correlated time series, on the other hand, refer to variables whose movements are typically in the same direction, but their spread in nonstationary.

What's the importance of stationarity?

We firstly determine the stationarity for our multivariate time series by performing the Johansen's cointegration test. (Johansen) We do not utilize the Augmented Dickey Fuller (ADF) test for each univariate time series in our data, since we are not interested in the stationarity of each variable in a multivariate time series. As mentioned previously, however, we understand that order flow and volume are univariately stationary (Cont et al).

The null hypothesis for Johansen's cointegration test is that the number of cointegrating relationships is less than the number of variables in our time series, while the alternative is that the number of cointegrating relationships equals the number of variables in our time series. By rejecting the null hypothesis at a specified significance level, we accept that the multivariate time series is stationary with that significance. We choose to perform the test at the 99% significance level.

In [12]:
from statsmodels.tsa.vector_ar.vecm import coint_johansen

/Users/ajcutuli/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
/Users/ajcutuli/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,


In [13]:
jres = coint_johansen(of_data.iloc[:,:-5], det_order=0, k_ar_diff=1)

/Users/ajcutuli/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/vector_ar/vecm.py:577: HypothesisTestWarning: Critical values are only available for time series with 12 variables at most.
  warnings.warn("Critical values are only available for time series "


In [14]:
jres.trace_stat

array([621935.45490447, 581452.20827874, 544620.0447818 , 509030.66011491,
       475135.45164202, 442163.79809793, 409749.2706151 , 377835.58947009,
       346473.16169092, 315628.48501215, 284988.63912294, 255111.994928  ,
       225514.45328588, 196108.95905264, 167017.82769008, 138296.79740734,
       109852.00670167,  81872.88978133,  54150.81191085,  26972.59291661])

In [15]:
jres.trace_stat_crit_vals[:,2]

array([     nan,      nan,      nan,      nan,      nan,      nan,
            nan,      nan, 351.215 , 300.2821, 253.2526, 210.0366,
       171.0905, 135.9825, 104.9637,  77.8202,  54.6815,  35.4628,
        19.9349,   6.6349])

The generated trace statistics are far greater than the critical values at the 99% significance level, so we can reject the null hypothesis at the 99% significance level. That is, we are at least 99% confident that the number of cointegrating relationships is equal to the number of variables in our multivariate time series, so we consider the multivariate time series data to be stationary.

### Remark on Autocorrelation

Classically, the order of the autoregressive process is what we use to control the lag parameter of the deep learning model. For a univariate model, this can be determined from the time series data with a partial autocorrelation test. However, for a multivariate model such as ours, the estimation process is different since we must account for the cross-correlations between the 20 variables.

Now, the articles from which we adopt the deep learning model offer a lag parameter of 100 timestamps. However, the mean squared error of a 1-step ahead prediction increases with respect to the order, so we choose to estimate it statistically and select it as our window length. We can do so on the basis that if a process has order *p*, then it has order *p+1*.

The `VAR` class of the `statsmodels` package assumes that the passed time series are stationary, so we can use it for our estimation of the order of our process.

Choice of lag order can be a difficult problem. Standard analysis employs likelihood test or information criteria-based order selection.

Pros and cons of increasing/decreasing lag parameter?

In [16]:
from statsmodels.tsa.api import VAR

In [17]:
model = VAR(of_data.iloc[:,:-5])
results = model.fit(maxlags=100, ic='aic')
lag_order = results.k_ar
print(lag_order)

15


### Splitting the Time Series

We split the training and test set by using the first 80% of the time series and the remaining 20% for the test set. Note that the test set must be in the future of the training set to avoid look-ahead bias.

In [18]:
# Train test split
train_weight = 0.8
of_train, of_test = np.array([None]), np.array([None])
of_train = of_data.iloc[:int(len(of_data)*train_weight)]
of_test = of_data.iloc[int(len(of_data)*train_weight):]

### Scaling

Standardization of the data is important to avoid potential scaling difficulties in the fitting of the model. When there is more than one feature (covariate), scaling avoids one feature dominating over another due to disparate scales.

To avoid introducing a look-ahead bias into the prediction, we must re-scale the training data without knowledge of the test set. Hence, we will simply standardize the training set using the mean and standard deviation of the training set and not the whole time series. Additionally, to avoid introducing a systematic bias into test set, we use the identical normalization for the test set - the mean and standard deviation of the training set are used to normalize the test set.

In [19]:
# Standardize data
# note that for a multivariate time series, you would need to scale 
# each variable by its own mean and standard deviation in the training set
for col in of_train.columns[:-5]:
    mu = np.float(of_train[col].mean())
    sigma = np.float(of_train[col].std())
    stdize_input = lambda x: (x - mu) / sigma
    of_train[col] = of_train[col].apply(stdize_input)
    of_test[col] = of_test[col].apply(stdize_input)

<ipython-input-19-dc6e8278564a>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  of_train[col] = of_train[col].apply(stdize_input)
<ipython-input-19-dc6e8278564a>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  of_test[col] = of_test[col].apply(stdize_input)


### Data Formatting

To perform our sequential learning task, we must tensorize our data to create a time series that will serve as the inputs to the network.

Let's define the following function for reshaping the data into a return prediction format for the 5 time horizons of the original set.

In [20]:
# ready data for training:
# 1. sample_size=lag_order: the most recent lag_order updates, number of lags, i.e. sequence length
# 2. feature_num=20: 20 features per time stamp, number of variables in the series
# 3. target_num=5: relative changes for the next 1,2,3,5 and 10 events(5 in total), number of forecasting horizons
def get_lagged_features(data, sample_size=lag_order, feature_num=20, target_num=5):
    data = data.values
    shape = data.shape
    X = np.zeros((shape[0]-sample_size, sample_size, feature_num))
    Y = np.zeros(shape=(shape[0]-sample_size, target_num))
    for i in range(shape[0]-sample_size):
        X[i] = data[i:i+sample_size,:feature_num]# take the first 20 columns as features
        Y[i] = data[i+sample_size-1,-target_num:]# take the last 5 columns as labels
    X = X.reshape(X.shape[0], sample_size, feature_num, 1)# add the 4th dimension: 1 channel
    
    # "Benchmark dataset for mid-price forecasting of limit order book data with machine learning"
    # labels 1: equal to or greater than 0.002
    # labels 2: -0.00199 to 0.00199
    # labels 3: smaller or equal to -0.002
    # Y=Y+1 relabels as 0,1,2
    Y = Y+1
    return X,Y

In [21]:
train_X, train_Y = get_lagged_features(of_train)
train_Y = train_Y.astype(int)

In [22]:
print(train_X.shape)

(61157, 15, 20, 1)


### Model Specification

In [23]:
from tensorflow.keras import Input
from tensorflow.keras import layers
from tensorflow.keras.layers import LSTM
from tensorflow.keras.models import Model
import keras.initializers
from tensorflow.keras.layers import Dropout
import tensorflow as tf

In [24]:
def CNN_LSTM(dropout = 0.0, seed = 0):
    # the size of a single input is (lag_order,20). Note that we statistically generated a lag order of 15.
    input_tensor = Input(shape=(lag_order,20,1))

    # convolutional filter is (1,2) with stride of (1,2)
    layer_x = layers.Conv2D(16, (1,2), strides=(1,2), kernel_initializer=keras.initializers.glorot_uniform(seed), bias_initializer=keras.initializers.glorot_uniform(seed))(input_tensor)
    layer_x = layers.LeakyReLU(alpha=0.01)(layer_x)
    layer_x = layers.Conv2D(16, (4,1), padding='same', kernel_initializer=keras.initializers.glorot_uniform(seed), bias_initializer=keras.initializers.glorot_uniform(seed))(layer_x)
    layer_x = layers.LeakyReLU(alpha=0.01)(layer_x)
    layer_x = layers.Conv2D(16, (4,1), padding='same', kernel_initializer=keras.initializers.glorot_uniform(seed), bias_initializer=keras.initializers.glorot_uniform(seed))(layer_x)
    layer_x = layers.LeakyReLU(alpha=0.01)(layer_x)

    layer_x = layers.Conv2D(16, (1,10), kernel_initializer=keras.initializers.glorot_uniform(seed), bias_initializer=keras.initializers.glorot_uniform(seed))(layer_x)
    layer_x = layers.LeakyReLU(alpha=0.01)(layer_x)
    layer_x = layers.Conv2D(16, (4,1), padding='same', kernel_initializer=keras.initializers.glorot_uniform(seed), bias_initializer=keras.initializers.glorot_uniform(seed))(layer_x)
    layer_x = layers.LeakyReLU(alpha=0.01)(layer_x)
    layer_x = layers.Conv2D(16, (4,1), padding='same', kernel_initializer=keras.initializers.glorot_uniform(seed), bias_initializer=keras.initializers.glorot_uniform(seed))(layer_x)
    layer_x = layers.LeakyReLU(alpha=0.01)(layer_x)

    # Inception Module
    tower_1 = layers.Conv2D(32, (1,1), padding='same', kernel_initializer=keras.initializers.glorot_uniform(seed), bias_initializer=keras.initializers.glorot_uniform(seed))(layer_x)
    tower_1 = layers.LeakyReLU(alpha=0.01)(tower_1)
    tower_1 = layers.Conv2D(32, (3,1), padding='same', kernel_initializer=keras.initializers.glorot_uniform(seed), bias_initializer=keras.initializers.glorot_uniform(seed))(tower_1)
    tower_1 = layers.LeakyReLU(alpha=0.01)(tower_1)

    tower_2 = layers.Conv2D(32, (1,1), padding='same', kernel_initializer=keras.initializers.glorot_uniform(seed), bias_initializer=keras.initializers.glorot_uniform(seed))(layer_x)
    tower_2 = layers.LeakyReLU(alpha=0.01)(tower_2)
    tower_2 = layers.Conv2D(32, (5,1), padding='same', kernel_initializer=keras.initializers.glorot_uniform(seed), bias_initializer=keras.initializers.glorot_uniform(seed))(tower_2)
    tower_2 = layers.LeakyReLU(alpha=0.01)(tower_2)  

    tower_3 = layers.MaxPooling2D((3,1), padding='same', strides=(1,1))(layer_x)
    tower_3 = layers.Conv2D(32, (1,1), padding='same', kernel_initializer=keras.initializers.glorot_uniform(seed), bias_initializer=keras.initializers.glorot_uniform(seed))(tower_3)
    tower_3 = layers.LeakyReLU(alpha=0.01, )(tower_3)

    layer_x = layers.concatenate([tower_1, tower_2, tower_3], axis=-1)

    # concatenate features of tower_1, tower_2, tower_3
    layer_x = layers.Reshape((lag_order,96))(layer_x)
    
    # insert dropout layer
    layer_x = Dropout(dropout)(layer_x)
    
    # 64 LSTM units
    layer_x = LSTM(64, kernel_initializer=keras.initializers.glorot_uniform(0), bias_initializer=keras.initializers.glorot_uniform(0), recurrent_initializer=keras.initializers.orthogonal(0))(layer_x)
    # The last output layer uses a softmax activation function
    output = layers.Dense(3, activation='softmax', kernel_initializer=keras.initializers.glorot_uniform(0), bias_initializer=keras.initializers.glorot_uniform(0))(layer_x)
    model = Model(input_tensor, output)
    opt = tf.keras.optimizers.Adam(lr=0.01, epsilon=1) # learning rate and epsilon are the same as paper DeepLOB
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model
    

In [25]:
model_params = {
    'model': None, 'function': CNN_LSTM, 'dropout': 0.0, 'label':'CNN_LSTM'
}

### Cross-validation

any model that has been fitted under a Box–Jenkins approach needs to be assessed out-of-sample by time series cross-validation

As advocated for in BDLOB, we utilize dropout variational inference as a stochastic regularizer in the deep neural network. We use grid search to detect an appropriate dropout rate. We insert this dropout layer after the Inception Module.

We apply early stopping for all models to avoid overfitting, terminating training when validation loss has not decreased for five consecutive epochs.

In the time series model, observations are strongly related to their recent histories. Hence, the ordering of our data matters, so we can't apply the typical K-fold cross-validation that is applied in cross-sectional models. In prediction models over time series data, no future observations can be used in the training set. Instead, a sliding window must be used to train and predict out-of-sample over multiple repetitions to allow for parameter tuning

Do we fix the length of the sliding training window or do we allow it to "telescope" in time? There are pros and cons to choosing the latter; it includes more observations for training, but parameter confidence loses interpretability due to the loss of sample size control.

In [26]:
from keras.callbacks import EarlyStopping

In [27]:
es = EarlyStopping(monitor='loss', mode='min', verbose=1, patience=5, min_delta=3e-5, restore_best_weights=True)

In [28]:
from sklearn.model_selection import KFold, TimeSeriesSplit, GridSearchCV
from keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.keras.utils import to_categorical

In [29]:
max_epochs = 100
batch_size = 32

In [ ]:
dropout = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
    
# A dictionary containing a list of values to be iterated through
# for each parameter of the model included in the search
param_grid = {'dropout': dropout}
    
# In the kth split, TimeSeriesSplit returns first k folds
# as training set and the (k+1)th fold as test set.
tscv = TimeSeriesSplit(n_splits = 5)
    
# A grid search is performed for each of the models, and the parameter set which
# performs best over all the cross-validation splits is saved in the `params` dictionary
model = KerasRegressor(build_fn=model_params['function'], epochs=max_epochs, 
                        batch_size=batch_size, verbose=2)
grid = GridSearchCV(estimator=model, param_grid=param_grid, 
                    cv=tscv, n_jobs=1, verbose=1)
train_y5 = to_categorical(train_Y[:,0]) # y is the next event's mid price (k=1)
grid_result = grid.fit(train_X, train_y5, callbacks=[es])
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
        
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params_ = grid_result.cv_results_['params']
for mean, stdev, param_ in zip(means, stds, params_):
    print("%f (%f) with %r" % (mean, stdev, param_))
            
    model_params['dropout'] = grid_result.best_params_['dropout']

<ipython-input-85-3df6be371a79>:13: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  model = KerasRegressor(build_fn=model_params['function'], epochs=max_epochs,
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 5 folds for each of 5 candidates, totalling 25 fits


/Users/ajcutuli/opt/anaconda3/lib/python3.8/site-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epoch 1/2000
11/11 - 7s - loss: 1.0901 - accuracy: 0.4107 - 7s/epoch - 625ms/step
Epoch 2/2000
11/11 - 3s - loss: 1.0899 - accuracy: 0.4107 - 3s/epoch - 306ms/step
Epoch 3/2000
11/11 - 4s - loss: 1.0896 - accuracy: 0.4107 - 4s/epoch - 327ms/step
Epoch 4/2000
11/11 - 3s - loss: 1.0893 - accuracy: 0.4107 - 3s/epoch - 303ms/step
Epoch 5/2000
11/11 - 3s - loss: 1.0889 - accuracy: 0.4107 - 3s/epoch - 293ms/step
Epoch 6/2000
11/11 - 3s - loss: 1.0886 - accuracy: 0.4107 - 3s/epoch - 310ms/step
Epoch 7/2000
11/11 - 3s - loss: 1.0882 - accuracy: 0.4107 - 3s/epoch - 307ms/step
Epoch 8/2000
11/11 - 4s - loss: 1.0879 - accuracy: 0.4107 - 4s/epoch - 367ms/step
Epoch 9/2000
11/11 - 4s - loss: 1.0876 - accuracy: 0.4107 - 4s/epoch - 321ms/step
Epoch 10/2000
11/11 - 4s - loss: 1.0873 - accuracy: 0.4107 - 4s/epoch - 322ms/step
Epoch 11/2000
11/11 - 6s - loss: 1.0870 - accuracy: 0.4107 - 6s/epoch - 530ms/step
Epoch 12/2000
11/11 - 6s - loss: 1.0868 - accuracy: 0.4107 - 6s/epoch - 539ms/step
Epoch 13/2000

Epoch 100/2000
11/11 - 3s - loss: 1.0853 - accuracy: 0.4107 - 3s/epoch - 297ms/step
Epoch 101/2000
11/11 - 3s - loss: 1.0853 - accuracy: 0.4107 - 3s/epoch - 316ms/step
Epoch 102/2000
11/11 - 3s - loss: 1.0853 - accuracy: 0.4107 - 3s/epoch - 296ms/step
Epoch 103/2000
11/11 - 3s - loss: 1.0853 - accuracy: 0.4107 - 3s/epoch - 300ms/step
Epoch 104/2000
11/11 - 3s - loss: 1.0853 - accuracy: 0.4107 - 3s/epoch - 295ms/step
Epoch 105/2000
11/11 - 3s - loss: 1.0853 - accuracy: 0.4107 - 3s/epoch - 295ms/step
Epoch 106/2000
11/11 - 4s - loss: 1.0853 - accuracy: 0.4107 - 4s/epoch - 371ms/step
Epoch 107/2000
11/11 - 3s - loss: 1.0853 - accuracy: 0.4107 - 3s/epoch - 301ms/step
Epoch 108/2000
11/11 - 3s - loss: 1.0853 - accuracy: 0.4107 - 3s/epoch - 295ms/step
Epoch 109/2000
11/11 - 3s - loss: 1.0853 - accuracy: 0.4107 - 3s/epoch - 295ms/step
Epoch 110/2000
11/11 - 3s - loss: 1.0853 - accuracy: 0.4107 - 3s/epoch - 301ms/step
Epoch 111/2000
11/11 - 3s - loss: 1.0853 - accuracy: 0.4107 - 3s/epoch - 296

...

Epoch 342/2000
51/51 - 20s - loss: 0.6388 - accuracy: 0.7378 - 20s/epoch - 386ms/step
Epoch 343/2000
51/51 - 20s - loss: 0.6388 - accuracy: 0.7389 - 20s/epoch - 388ms/step
Epoch 344/2000
51/51 - 19s - loss: 0.6381 - accuracy: 0.7387 - 19s/epoch - 379ms/step
Epoch 345/2000
51/51 - 19s - loss: 0.6387 - accuracy: 0.7389 - 19s/epoch - 378ms/step
Epoch 346/2000
51/51 - 17s - loss: 0.6390 - accuracy: 0.7389 - 17s/epoch - 332ms/step
Epoch 347/2000
51/51 - 17s - loss: 0.6389 - accuracy: 0.7400 - 17s/epoch - 341ms/step
Epoch 348/2000
51/51 - 17s - loss: 0.6383 - accuracy: 0.7392 - 17s/epoch - 335ms/step
Epoch 349/2000
51/51 - 17s - loss: 0.6376 - accuracy: 0.7377 - 17s/epoch - 328ms/step
Epoch 350/2000
51/51 - 17s - loss: 0.6389 - accuracy: 0.7382 - 17s/epoch - 330ms/step
Epoch 351/2000
51/51 - 17s - loss: 0.6380 - accuracy: 0.7400 - 17s/epoch - 337ms/step
Epoch 352/2000
51/51 - 17s - loss: 0.6387 - accuracy: 0.7386 - 17s/epoch - 326ms/step
Epoch 353/2000
51/51 - 17s - loss: 0.6373 - accuracy: 

Epoch 438/2000
51/51 - 19s - loss: 0.6280 - accuracy: 0.7441 - 19s/epoch - 363ms/step
Epoch 439/2000
51/51 - 18s - loss: 0.6278 - accuracy: 0.7440 - 18s/epoch - 350ms/step
Epoch 440/2000
51/51 - 18s - loss: 0.6280 - accuracy: 0.7453 - 18s/epoch - 347ms/step
Epoch 441/2000
51/51 - 18s - loss: 0.6281 - accuracy: 0.7440 - 18s/epoch - 346ms/step
Epoch 442/2000
51/51 - 19s - loss: 0.6279 - accuracy: 0.7451 - 19s/epoch - 365ms/step
Epoch 443/2000
51/51 - 17s - loss: 0.6274 - accuracy: 0.7450 - 17s/epoch - 343ms/step
Epoch 444/2000
51/51 - 18s - loss: 0.6291 - accuracy: 0.7445 - 18s/epoch - 355ms/step
Epoch 445/2000
51/51 - 19s - loss: 0.6274 - accuracy: 0.7451 - 19s/epoch - 365ms/step
Epoch 446/2000
51/51 - 18s - loss: 0.6283 - accuracy: 0.7444 - 18s/epoch - 345ms/step
Epoch 447/2000
51/51 - 18s - loss: 0.6270 - accuracy: 0.7442 - 18s/epoch - 353ms/step
Epoch 448/2000
51/51 - 18s - loss: 0.6272 - accuracy: 0.7460 - 18s/epoch - 354ms/step
Epoch 449/2000
51/51 - 19s - loss: 0.6269 - accuracy: 

Epoch 534/2000
51/51 - 18s - loss: 0.6204 - accuracy: 0.7498 - 18s/epoch - 348ms/step
Epoch 535/2000
51/51 - 17s - loss: 0.6198 - accuracy: 0.7496 - 17s/epoch - 337ms/step
Epoch 536/2000
51/51 - 20s - loss: 0.6203 - accuracy: 0.7493 - 20s/epoch - 384ms/step
Epoch 537/2000
51/51 - 19s - loss: 0.6194 - accuracy: 0.7494 - 19s/epoch - 368ms/step
Epoch 538/2000
51/51 - 18s - loss: 0.6196 - accuracy: 0.7485 - 18s/epoch - 356ms/step
Epoch 539/2000
51/51 - 17s - loss: 0.6204 - accuracy: 0.7485 - 17s/epoch - 339ms/step
Epoch 540/2000
51/51 - 17s - loss: 0.6203 - accuracy: 0.7491 - 17s/epoch - 340ms/step
Epoch 541/2000
51/51 - 19s - loss: 0.6194 - accuracy: 0.7496 - 19s/epoch - 367ms/step
Epoch 542/2000
51/51 - 19s - loss: 0.6195 - accuracy: 0.7492 - 19s/epoch - 365ms/step
Epoch 543/2000
51/51 - 20s - loss: 0.6195 - accuracy: 0.7493 - 20s/epoch - 389ms/step
Epoch 544/2000
51/51 - 19s - loss: 0.6183 - accuracy: 0.7495 - 19s/epoch - 367ms/step
Epoch 545/2000
51/51 - 21s - loss: 0.6191 - accuracy: 

Epoch 630/2000
51/51 - 17s - loss: 0.6136 - accuracy: 0.7511 - 17s/epoch - 333ms/step
Epoch 631/2000
51/51 - 17s - loss: 0.6125 - accuracy: 0.7524 - 17s/epoch - 330ms/step
Epoch 632/2000
51/51 - 17s - loss: 0.6145 - accuracy: 0.7508 - 17s/epoch - 328ms/step
Epoch 633/2000
51/51 - 17s - loss: 0.6130 - accuracy: 0.7524 - 17s/epoch - 336ms/step
Epoch 634/2000
51/51 - 17s - loss: 0.6135 - accuracy: 0.7517 - 17s/epoch - 339ms/step
Epoch 635/2000
51/51 - 17s - loss: 0.6143 - accuracy: 0.7512 - 17s/epoch - 339ms/step
Epoch 636/2000
51/51 - 17s - loss: 0.6136 - accuracy: 0.7526 - 17s/epoch - 329ms/step
Epoch 637/2000
51/51 - 17s - loss: 0.6151 - accuracy: 0.7518 - 17s/epoch - 328ms/step
Epoch 638/2000
51/51 - 17s - loss: 0.6139 - accuracy: 0.7512 - 17s/epoch - 333ms/step
Epoch 639/2000
51/51 - 17s - loss: 0.6134 - accuracy: 0.7516 - 17s/epoch - 330ms/step
Epoch 640/2000
51/51 - 17s - loss: 0.6131 - accuracy: 0.7515 - 17s/epoch - 331ms/step
Epoch 641/2000
51/51 - 17s - loss: 0.6128 - accuracy: 

Epoch 726/2000
51/51 - 19s - loss: 0.6083 - accuracy: 0.7535 - 19s/epoch - 378ms/step
Epoch 727/2000
51/51 - 21s - loss: 0.6079 - accuracy: 0.7547 - 21s/epoch - 405ms/step
Epoch 728/2000
51/51 - 23s - loss: 0.6082 - accuracy: 0.7543 - 23s/epoch - 448ms/step
Epoch 729/2000
51/51 - 20s - loss: 0.6079 - accuracy: 0.7557 - 20s/epoch - 396ms/step
Epoch 730/2000
51/51 - 21s - loss: 0.6079 - accuracy: 0.7546 - 21s/epoch - 406ms/step
Epoch 731/2000
51/51 - 21s - loss: 0.6075 - accuracy: 0.7563 - 21s/epoch - 413ms/step
Epoch 732/2000
51/51 - 22s - loss: 0.6082 - accuracy: 0.7546 - 22s/epoch - 426ms/step
Epoch 733/2000
51/51 - 19s - loss: 0.6069 - accuracy: 0.7558 - 19s/epoch - 376ms/step
Epoch 734/2000
51/51 - 18s - loss: 0.6075 - accuracy: 0.7560 - 18s/epoch - 349ms/step
Epoch 735/2000
51/51 - 18s - loss: 0.6078 - accuracy: 0.7545 - 18s/epoch - 361ms/step
Epoch 736/2000
51/51 - 21s - loss: 0.6078 - accuracy: 0.7543 - 21s/epoch - 407ms/step
Epoch 737/2000
51/51 - 20s - loss: 0.6071 - accuracy: 

### Training the Model

In [30]:
model = model_params['function'](dropout = model_params['dropout'])
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 15, 20, 1)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 15, 10, 16)   48          ['input_1[0][0]']                
                                                                                                  
 leaky_re_lu (LeakyReLU)        (None, 15, 10, 16)   0           ['conv2d[0][0]']                 
                                                                                                  
 conv2d_1 (Conv2D)              (None, 15, 10, 16)   1040        ['leaky_re_lu[0][0]']            
                                                                                              

/Users/ajcutuli/opt/anaconda3/lib/python3.8/site-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [35]:
train_X

array([[[[  0.05076311],
         [ -0.13890861],
         [  0.04294524],
         ...,
         [ -0.0952797 ],
         [  0.11131208],
         [ -0.06374609]],

        [[  0.05076311],
         [  0.18984078],
         [  0.04294524],
         ...,
         [  0.13299181],
         [  0.0309149 ],
         [  0.23129306]],

        [[  0.05076311],
         [  0.28697128],
         [  0.04294524],
         ...,
         [ -0.0952797 ],
         [ -0.05275045],
         [ -0.06374609]],

        ...,

        [[  0.05076311],
         [  0.03791871],
         [  0.04294524],
         ...,
         [  0.02582898],
         [  0.0309149 ],
         [  0.02693409]],

        [[  0.05076311],
         [  0.03791871],
         [  0.04294524],
         ...,
         [  0.02582898],
         [  0.0309149 ],
         [  0.02693409]],

        [[  0.05076311],
         [  0.03791871],
         [  0.04294524],
         ...,
         [  0.02582898],
         [  0.0309149 ],
         [  0.026

In [31]:
train_y5 = to_categorical(train_Y[:,0], num_classes=3) # y is the next event's mid price (k=1)
model.fit(train_X, train_y5, epochs=max_epochs, batch_size=batch_size, callbacks=[es])
model_params['model'] = model

Epoch 1/100
1912/1912 [==============================] - 45s 22ms/step - loss: 1.0410 - accuracy: 0.5000
Epoch 2/100
1912/1912 [==============================] - 43s 23ms/step - loss: 0.9571 - accuracy: 0.5450
Epoch 3/100
1351/1912 [====================>.........] - ETA: 14s - loss: 0.7707 - accuracy: 0.6727

KeyboardInterrupt: 

In [ ]:
test_X, test_Y = get_lagged_features(of_test)
test_Y = test_Y.astype(int)

test_y5 = to_categorical(test_Y[:,0])
model.evaluate(test_X, test_y5)

In [ ]:
from sklearn.metrics import log_loss

In [ ]:
model = model_params['model']
    
params[key]['pred_train'] = model.predict(train_X, verbose=1)
params[key]['CE_train'] = log_loss(train_y5, model_params['pred_train'])
    
params[key]['pred_test'] = model.predict(test_X, verbose=1) 
params[key]['CE_test'] = log_loss(test_y5, model_params['pred_test'])

In [ ]:
max_pts = 10**4 # maximum number of  points to plot per series

fig = plt.figure(figsize=(15,8))
x_vals = np.arange(len(train_y5))

y_vals = model_params['pred_train']
label = model_params['label'] + ' (train CE: %.2e)' % model_params['CE_train']
plt.plot(x_vals, y_vals, c='blue', label=label, lw=1)

plt.plot(x_vals, train_y5, c="black", label="Observed", lw=1)
plt.xlim(x_vals.min(), x_vals.max())
plt.xlabel('Time (ticks)', fontsize=14)
plt.ylabel('$\hat{Y}$', rotation=0, fontsize=14)
plt.legend(loc="best", fontsize=12)
plt.title('Observed vs Model Outputs (Training)', fontsize=16);

In [ ]:
fig = plt.figure(figsize=(15,8))
x_vals = np.arange(len(train_y5))

y_vals = model_params['pred_train'] - train_y5
label = model_params['label'] + ' (train CE: %.2e)' % model_params['CE_train']
plt.plot(x_vals, y_vals, c='blue', label=label, lw=1)

plt.axhline(0, linewidth=0.8)
plt.xlim(x_vals.min(), x_vals.max())
plt.xlabel('Time (ticks)', fontsize=14)
plt.ylabel('$\hat{Y}-Y$', fontsize=14)
plt.legend(loc="best", fontsize=12)
plt.title('Observed vs Model Error (Training)', fontsize=16);

In [ ]:
fig = plt.figure(figsize=(15,8))
x_vals = len(train_y5) + np.arange(len(test_y5))

y_vals = model_params['pred_test'][l:u:ds]
label = model_params['label'] + ' (test CE: %.2e)' % model_params['CE_test']
plt.plot(x_vals, y_vals, c='blue', label=label, lw=1)

plt.plot(x_vals, test_y5[l:u:ds], c="black", label="Observed", lw=1)
plt.xlim(x_vals.min(), x_vals.max())
plt.xlabel('Time (ticks)', fontsize=14)
plt.ylabel('$\hat{Y}$', rotation=0, fontsize=14)
plt.legend(loc="best", fontsize=12)
plt.title('Observed vs Model Outputs (Testing)', fontsize=16);

In [ ]:
fig = plt.figure(figsize=(15,8))
x_vals = len(train_y5) + np.arange(len(test_y5))

y_vals = model_params['pred_test'] - test_y5
label = model_params['label'] + ' (test CE: %.2e)' % model_params['CE_test']
plt.plot(x_vals, y_vals, c='blue', label=label, lw=1)

plt.axhline(0, linewidth=0.8)
plt.xlim(x_vals.min(), x_vals.max())
plt.xlabel('Time (ticks)', fontsize=14)
plt.ylabel('$\hat{Y}-Y$', fontsize=14)
plt.legend(loc="best", fontsize=12)
plt.title('Observed vs Model Error (Testing)', fontsize=16);

## Building a Trading Signal

what happens if we go long every time the model predicts a positive return and likewise go short if the model predicts a negative return.

if our model were to predict a positive return for the next second, we would immediately buy at the close and be filled at the closing price. We would then close out our position after the second elapsed, again, getting filled at the next close to produce a return.

This is essentially the "Normal trading strategy" that is described in (Zhang et al)


## Shortfalls of Deep Learning

Deep neural networks as a "black box"

"Offline" version of learning

## Conclusion

## References

[1] Zhang, Zihao, Stefan Zohren, and Stephen Roberts (2019). “DeepLOB: Deep Convolutional Neural Networks for Limit Order Books”. In: *IEEE Transactions On Signal Processing* 67.11, pp. 3001–3012.<br>
[2] Cont, Rama, Arseniy Kukanov, and Sasha Stoikov (2014). “The Price Impact Of
Order Book Events”. In: *Journal Of Financial Econometrics* 12.1, pp. 47–88.<br>
[3] Kolm, Petter, Jeremy Turiel, and Nicholas Westray (2021). "Deep Order Flow Imbalance: Extracting Alpha At Multiple Horizons From The Limit Order Book". In: *Available at SSRN 3900141*.<br>
[] Ntakaris, Adamantios, Martin Magris, Juho Kanniainen, Moncef Gabbouj, and Alexandros Iosifidis (2018). “Benchmark Dataset For Mid-Price Forecasting Of Limit Order Book Data With Machine Learning Methods”. In: *Journal of Forecasting* 37.8, pp. 852–866.<br>
[] Szegedy, Christian, Wei Liu, Yangqing Jia, Pierre Sermanet, Scott Reed, Dragomir Anguelov, Dumitru Erhan, Vincent Vanhoucke, and Andrew Rabinovich (2015). “Going Deeper With Convolutions”. In: *Proceedings Of The IEEE Conference On Computer Vision And Pattern Recognition*, pp. 1–9.<br>
[] Zhang, Zihao, Stefan Zohren, and Stephen Roberts (2018). “BDLOB: Bayesian Deep Convolutional Neural Networks For Limit Order Books”. In: *arXiv preprint arXiv:1811.10041*.<br>
[] Dixon, Matthew F., Igor Halperin, and Paul Bilokon (2020). *Machine Learning in Finance: From Theory to Practice*. Springer.<br>
[] Tsay, Ruey S. (2010). *Analysis of Financial Time Series* (3rd ed.). Wiley.<br>
[] Lütkepohl, Helmut (2005). *New Introduction to Multiple Time Series Analysis*. Springer.<br> 
[] Yang, Kiyoung, and Cyrus Shahabi (2005). "On the Stationarity of Multivariate Time Series for Correlation-Based Data Analysis". In: *Fifth IEEE International Conference on Data Mining*, pp. 1-4.<br>
[] Johansen, Søren (1995). *Likelihood-Based Inference in Cointegrated Vector Autoregressive Models*. Oxford University Press.<br>